<a href="https://colab.research.google.com/github/honoreade/DeepLearning/blob/main/tf_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the pretrained MobileNetV2 model
model = MobileNetV2(weights='imagenet')

# Load and preprocess the image
img_path = '/content/drive/MyDrive/Colab Notebooks/images.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Predict and decode predictions
preds = model.predict(img_array)
decoded_preds = decode_predictions(preds, top=5)[0]

# Print predictions to see if "banana" is one of the top predictions
for _, class_name, score in decoded_preds:
    print(f"{class_name}: {score:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
great_white_shark: 0.9219
tiger_shark: 0.0188
barracouta: 0.0080
sturgeon: 0.0023
killer_whale: 0.0020


#  Making inferences on MobileNetV2, ResNet50, InceptionV3

In [ ]:
# prompt: make the fuction for testing multiple vision pretained model at once
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2, ResNet50, InceptionV3
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as preprocess_mobilenet
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_inception
from tensorflow.keras.preprocessing import image
import numpy as np

def test_multiple_models(image_path):
    """Tests multiple pretrained vision models on a given image.

    Args:
        image_path: The path to the image file.
    """

    models = {
        "MobileNetV2": (MobileNetV2(weights='imagenet'), preprocess_mobilenet, (224, 224)),
        "ResNet50": (ResNet50(weights='imagenet'), preprocess_resnet, (224, 224)),
        "InceptionV3": (InceptionV3(weights='imagenet'), preprocess_inception, (299, 299)),
    }

    for model_name, (model, preprocess_func, target_size) in models.items():
      print(f"\nTesting with {model_name}:")
      try:
        img = image.load_img(image_path, target_size=target_size)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_func(img_array)

        preds = model.predict(img_array)

        if model_name == "MobileNetV2" or model_name == "ResNet50":
          decoded_preds = tf.keras.applications.mobilenet_v2.decode_predictions(preds, top=5)[0]
        elif model_name == "InceptionV3":
          decoded_preds = tf.keras.applications.inception_v3.decode_predictions(preds, top=5)[0]

        for _, class_name, score in decoded_preds:
            print(f"{class_name}: {score:.2f}")

      except Exception as e:
          print(f"Error processing {model_name}: {e}")


# Example usage (assuming you have mounted your Google Drive):
img_path = '/content/drive/MyDrive/Colab Notebooks/images.jpg'
test_multiple_models(img_path)



Testing with MobileNetV2:
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
great_white_shark: 0.92
tiger_shark: 0.02
barracouta: 0.01
sturgeon: 0.00
killer_whale: 0.00

Testing with ResNet50:
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
great_white_shark: 0.87
tiger_shark: 0.11
hammerhead: 0.00
killer_whale: 0.00
electric_ray: 0.00

Testing with InceptionV3:
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
great_white_shark: 0.83
tiger_shark: 0.13
hammerhead: 0.00
killer_whale: 0.00
electric_ray: 0.00


# **try RegNet by facebook ai**

In [ ]:
# prompt: try RegNet by facebook ai

import torch
from torchvision import models, transforms
from PIL import Image

# Load a pre-trained RegNet model
model = models.regnet_y_400mf(pretrained=True)
model.eval()  # Set model to evaluation mode

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to model's expected input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load and preprocess an image
image_path = '/content/drive/MyDrive/Colab Notebooks/images.jpg'  # Replace with your image path
image = Image.open(image_path).convert("RGB")
input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# Run inference
with torch.no_grad():
    output = model(input_tensor)

# Interpret the result
probabilities = torch.nn.functional.softmax(output[0], dim=0)
predicted_class = torch.argmax(probabilities).item()

# Print the result
print(f"Predicted class index: {predicted_class}")



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RegNet_Y_400MF_Weights.IMAGENET1K_V1`. You can also use `weights=RegNet_Y_400MF_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Predicted class index: 2


In [ ]:
import json
import urllib.request

# Download ImageNet class labels
url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
imagenet_classes = urllib.request.urlopen(url).read().decode("utf-8").split("\n")

# Get the predicted class name
predicted_class_name = imagenet_classes[predicted_class]

print(f"Predicted class: {predicted_class_name}")


Predicted class: great white shark


# **For NASNetLarge Vs NASNetMobile**

In [ ]:
from tensorflow.keras.applications import NASNetLarge, NASNetMobile
from tensorflow.keras.applications.nasnet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np

# Load pre-trained models
model_large = NASNetLarge(weights="imagenet")
model_mobile = NASNetMobile(weights="imagenet")

# Function to load and preprocess image with given target size
def preprocess_image(image_path, target_size):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Preprocess for both models
img_array_large = preprocess_image(img_path, (331, 331))  # For NASNetLarge
img_array_mobile = preprocess_image(img_path, (224, 224)) # For NASNetMobile

# Run inference
preds_large = model_large.predict(img_array_large)
preds_mobile = model_mobile.predict(img_array_mobile)

# Decode results
print("NASNetLarge Prediction:", decode_predictions(preds_large, top=3)[0])
print("NASNetMobile Prediction:", decode_predictions(preds_mobile, top=3)[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
NASNetLarge Prediction: [('n01484850', 'great_white_shark', 0.86783826), ('n01491361', 'tiger_shark', 0.035030294), ('n01494475', 'hammerhead', 0.0017078795)]
NASNetMobile Prediction: [('n01484850', 'great_white_shark', 0.63411146), ('n01491361', 'tiger_shark', 0.13114019), ('n01494475', 'hammerhead', 0.029718278)]
